<a href="https://colab.research.google.com/github/SuyashaaR/PSR/blob/master/suyashaa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
df = pd.read_excel("https://github.com/SuyashaaR/PSR/blob/master/PANDAS.xlsx?raw=true")

In [0]:
df.head()

,Enrolled,Age,Gender,Active SMP,Ads. SMP,Online Purchase 6M,R&R Purchase Decsions,Products OS,Decision,Useful ads,Annoying ads,Ads free,Related ads
0,Ph.D,22,Male,"Facebook, Linkedin, WhatsApp, YouTube, Quora","Facebook, Instagram, Linkedin, WhatsApp, YouTube",Between 3 to 6 times,No,"Clothes, Electronic gadgets, Computer products",Wait,4,1,Yes,Often
1,Masters',24,Male,"Facebook, Linkedin, WhatsApp, YouTube, Quora","Facebook, YouTube, others",Less than 3 times,No,"Electronic gadgets, Computer products, Tickets",Wait,5,1,Yes,Always
2,Bachelors',19,Male,"WhatsApp, YouTube","Linkedin, YouTube",Less than 3 times,Yes,"Books/CDs, Electronic gadgets, Tickets",Wait,3,1,Yes,Sometimes
3,Bachelors',20,Male,"Facebook, Instagram, WhatsApp, YouTube","Facebook, Instagram, YouTube",Less than 3 times,Maybe,"Clothes, Furnitures, Electronic gadgets",DEPENDS,3,4,Yes,Rarely
4,Bachelors',19,Male,"Twitter, Instagram, WhatsApp, YouTube, Quora, ...","Instagram, YouTube",Less than 3 times,Yes,"Clothes, Electronic gadgets, Computer products...",Buy immediately,2,4,No,Sometimes


In [0]:
output_cats = pd.Categorical(df["Online Purchase 6M"])

In [0]:
output_cats

[Between 3 to 6 times, Less than 3 times, Less than 3 times, Less than 3 times, Less than 3 times, ..., Less than 3 times, Less than 3 times, Less than 3 times, Less than 3 times, Less than 3 times]
Length: 172
Categories (4, object): [Between 3 to 6 times, Between 6 to 9 times, Less than 3 times,
                         More than 12 times]

In [0]:
df["output"] = output_cats.codes

In [0]:
df.drop(columns=["Online Purchase 6M"], inplace=True)

In [0]:
df.head()

,Enrolled,Age,Gender,Active SMP,Ads. SMP,R&R Purchase Decsions,Products OS,Decision,Useful ads,Annoying ads,Ads free,Related ads,output
0,Ph.D,22,Male,"Facebook, Linkedin, WhatsApp, YouTube, Quora","Facebook, Instagram, Linkedin, WhatsApp, YouTube",No,"Clothes, Electronic gadgets, Computer products",Wait,4,1,Yes,Often,0
1,Masters',24,Male,"Facebook, Linkedin, WhatsApp, YouTube, Quora","Facebook, YouTube, others",No,"Electronic gadgets, Computer products, Tickets",Wait,5,1,Yes,Always,2
2,Bachelors',19,Male,"WhatsApp, YouTube","Linkedin, YouTube",Yes,"Books/CDs, Electronic gadgets, Tickets",Wait,3,1,Yes,Sometimes,2
3,Bachelors',20,Male,"Facebook, Instagram, WhatsApp, YouTube","Facebook, Instagram, YouTube",Maybe,"Clothes, Furnitures, Electronic gadgets",DEPENDS,3,4,Yes,Rarely,2
4,Bachelors',19,Male,"Twitter, Instagram, WhatsApp, YouTube, Quora, ...","Instagram, YouTube",Yes,"Clothes, Electronic gadgets, Computer products...",Buy immediately,2,4,No,Sometimes,2


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()


In [0]:
df2 = pd.DataFrame(mlb.fit_transform(df["Active SMP"].str.replace(' ', '').str.split(',')),columns=mlb.classes_, index=df.index)


In [0]:
pd.DataFrame(mlb.fit_transform(df["Active SMP"].str.replace(' ', '').str.split(',')),columns=mlb.classes_, index=df.index).sum()

Facebook     141
Instagram    141
Linkedin      54
Quora         70
Snapchat      30
Telegram       5
Twitter       40
WhatsApp     162
YouTube      148
others         7
dtype: int64

In [0]:
df3 = pd.DataFrame(mlb.fit_transform(df["Ads. SMP"].str.replace(' ', '').str.split(',')),columns=mlb.classes_, index=df.index)

In [0]:
pd.DataFrame(mlb.fit_transform(df["Ads. SMP"].str.replace(' ', '').str.split(',')),columns=mlb.classes_, index=df.index).sum()

Facebook     146
Instagram    136
Linkedin      18
Quora         31
Snapchat      11
Twitter       16
WhatsApp      20
YouTube      143
others         3
dtype: int64

In [0]:
bins = [10, 20, 30, 40, 50]
group_names = ['10-19', '20-29', '30-39', '40-49']
age_categories = pd.cut(df['Age '], bins, labels=group_names)
df['age_categories'] = pd.cut(df['Age '], bins, labels=group_names)
age_categories
pd.value_counts(df['age_categories'])

20-29    87
10-19    80
30-39     3
40-49     1
Name: age_categories, dtype: int64

In [0]:
df.drop(columns=["Products OS", "Age ", "Enrolled", "Active SMP", "Ads. SMP"], inplace=True)
df["Decision"] = df["Decision"].str.strip()

In [0]:
df.join(df3)

,Gender,R&R Purchase Decsions,Decision,Useful ads,Annoying ads,Ads free,Related ads,output,age_categories,Facebook,Instagram,Linkedin,Quora,Snapchat,Twitter,WhatsApp,YouTube,others
0,Male,No,Wait,4,1,Yes,Often,0,20-29,1,1,1,0,0,0,1,1,0
1,Male,No,Wait,5,1,Yes,Always,2,20-29,1,0,0,0,0,0,0,1,1
2,Male,Yes,Wait,3,1,Yes,Sometimes,2,10-19,0,0,1,0,0,0,0,1,0
3,Male,Maybe,DEPENDS,3,4,Yes,Rarely,2,10-19,1,1,0,0,0,0,0,1,0
4,Male,Yes,Buy immediately,2,4,No,Sometimes,2,10-19,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Female,Yes,Wait,2,2,No,Always,2,20-29,1,0,0,0,0,0,0,1,0
168,Female,Maybe,DEPENDS,3,4,No,Often,2,20-29,1,1,0,0,0,0,0,1,0
169,Male,Yes,Wait,3,3,No,Sometimes,2,10-19,1,1,0,1,0,1,0,1,0
170,Male,Yes,Wait,3,1,Yes,Sometimes,2,10-19,1,1,0,1,0,0,0,1,0


In [0]:
df4 = pd.get_dummies(df).join(df3)

In [0]:
df4.head()

,Useful ads,Annoying ads,output,Gender _Female,Gender _Male,Gender _Prefer not to say,R&R Purchase Decsions_Maybe,R&R Purchase Decsions_No,R&R Purchase Decsions_Yes,Decision_Buy immediately,Decision_DEPENDS,Decision_Wait,Ads free_Maybe,Ads free_No,Ads free_Yes,Related ads_Always,Related ads_Often,Related ads_Rarely,Related ads_Sometimes,age_categories_10-19,age_categories_20-29,age_categories_30-39,age_categories_40-49,Facebook,Instagram,Linkedin,Quora,Snapchat,Twitter,WhatsApp,YouTube,others
0,4,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,1,1,0,0,0,1,1,0
1,5,1,2,0,1,0,0,1,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,1
2,3,1,2,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0
3,3,4,2,0,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,0,0,1,0
4,2,4,2,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0


In [0]:
import xgboost as xgb
from sklearn.model_selection import train_test_split


In [0]:
from xgboost import XGBClassifier
import numpy as np

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(df4.drop("output", axis=1), df4["output"], test_size=0.2)

In [0]:
linearRegressor = XGBClassifier()

In [0]:
linearRegressor.fit(np.array(X_train),np.array(Y_train))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
predictions = linearRegressor.predict(np.array(X_test))

In [0]:
from sklearn.metrics import accuracy_score

In [0]:
accuracy_score(Y_test, predictions)

0.7142857142857143